In [1]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Iris dataset 불러오기
X, y = datasets.load_iris(return_X_y=True)

# Train, Test 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 모델 하이퍼파라미터 정의
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# 모델 학습
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# test 데이터셋 Predict
y_pred = lr.predict(X_test)

# Metrics 계산
accuracy = accuracy_score(y_test, y_pred)

c:\Users\SSAFY\Desktop\TIL\11_deep_learning\03_c_mlflow\1_lecture\venvs\mlflow-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [2]:
# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

# 새로운 MLFlow 실험을 생성
mlflow.set_experiment("MLflow Quickstart")

# MLflow 실험 시작
with mlflow.start_run():
    # 하이퍼파라미터 로그
    mlflow.log_params(params)

    # loss, metric 로그
    mlflow.log_metric("accuracy", accuracy)

    # 태그 설정 (실행에 메모 남기기)
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # 입출력 형태 추론
    signature = infer_signature(X_train, lr.predict(X_train))

    # 모델 로그
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

2025/10/01 11:57:08 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart' does not exist. Creating a new experiment.
c:\Users\SSAFY\Desktop\TIL\11_deep_learning\03_c_mlflow\1_lecture\venvs\mlflow-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Successfully registered model 'tracking-quickstart'.
2025/10/01 11:57:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run colorful-dove-457 at: http://127.0.0.1:5000/#/experiments/2/runs/3d12989f63ba4f2cb5e5646ee5a1f07c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


Created version '1' of model 'tracking-quickstart'.


In [3]:
# 예측을 위해 일반 Python 함수 모델(pyfunc)로 모델을 다시 불러옵니다.
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
